# Lesson 3: Reflection and Blogpost Writing

In [23]:
"""
This script demonstrates a multi-agent writing workflow using AutoGen.
It includes:
1. A Writer agent → generates a blog post
2. A Critic agent → gives feedback
3. Specialized reviewers:
   - SEO Reviewer
   - Legal Reviewer
   - Ethics Reviewer
   - Meta Reviewer (aggregates all feedback)

Each agent has a very specific role and talks to the others to refine the
final output. This example shows how LLMs can collaborate in structured loops.
"""
"""
[1] Critic → Writer  (main chat starts)
         ↓
[2] Writer → Critic  (TRIGGER fires here!)
         ↓
[3] AutoGen spawns nested reviewer chats:
       SEO Reviewer → feedback
       Legal Reviewer → feedback
       Ethics Reviewer → feedback
       Meta Reviewer → aggregates feedback
         ↓
[4] Critic receives aggregated reviews
         ↓
[5] Critic → Writer (final summary or more feedback)
         ↓
[6] Conversation ends OR repeats"""

# =====================================================================
# 📦 Imports and Configuration
# =====================================================================
import autogen

# Configuration for the LLM used by all agents.
# A "tight prompt" setup means consistent model + deterministic output.
llm_config = {"model": "gpt-4o-mini"}

# The main writing task given to the Writer agent.
task = """
Write a concise, engaging blog post (with a title) about DeepLearning.AI.
The entire post must be under 100 words.
"""


# =====================================================================
# ✍️ Writer Agent
# =====================================================================
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config=llm_config,
    # This system message tightly defines the writer’s behavior.
    system_message=(
        "You are a professional blog writer. "
        "You produce polished, concise, and engaging blog posts WITH a title. "
        "You strictly follow the user's word limit. "
        "After receiving feedback, you revise the blog post and return ONLY the "
        "final improved version—no explanations, no extra comments."
    ),
)

# Writer generates the first draft
initial_reply = writer.generate_reply(messages=[{"role": "user", "content": task}])


# =====================================================================
# 🧠 Critic Agent — reviews and requests improvements
# =====================================================================
critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    # This function detects termination messages
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    system_message=(
        "You are a critic. Your job is to review the writer's blog post and "
        "give clear, constructive, actionable feedback to improve clarity, "
        "engagement, and quality. Avoid vague statements."
    ),
)

# First review loop: Critic talks to Writer
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,            # Writer + Critic exchange up to 2 turns
    summary_method="last_msg"
)


# =====================================================================
# 🔍 Specialized Review Agents
# These agents each focus on one dimension of evaluation.
# =====================================================================

# -------------------- SEO REVIEWER --------------------
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are an SEO expert. You review content for search engine ranking. "
        "Provide exactly 3 bullet points of improvement. Be concise and concrete. "
        "Start your review by clearly stating your role."
    ),
)

# -------------------- LEGAL REVIEWER --------------------
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are a legal compliance reviewer. You ensure that content is safe, "
        "non-misleading, and free from legal risk. Provide exactly 3 bullet points. "
        "Start your review by stating your role."
    ),
)

# -------------------- ETHICS REVIEWER --------------------
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are an ethics reviewer. You ensure the content is fair, neutral, "
        "and ethically responsible. Provide exactly 3 bullet points. "
        "Start your review by stating your role."
    ),
)

# -------------------- META REVIEWER --------------------
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message=(
        "You are a meta reviewer. You aggregate feedback from all other reviewers "
        "and provide a single, unified, final set of suggestions for the author."
    ),
)


# =====================================================================
# 🧩 Helper: Creates a message that passes content to reviewers
# =====================================================================
def reflection_message(recipient, messages, sender, config):
    """
    Returns the latest content from the Writer for reviewers to evaluate.
    This ensures reviewers always see the newest version.
    """
    latest = recipient.chat_messages_for_summary(sender)[-1]["content"]
    return f"Review the following content:\n\n{latest}"


# =====================================================================
# 📚 Register Nested Reviewer Chats
# Critic triggers these reviewers automatically after its review.
# =====================================================================
review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt":
                "Return ONLY a JSON object in this format: "
                "{'Reviewer': '', 'Review': ''}. "
                "Reviewer should be your role name."
        },
        "max_turns": 1,
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt":
                "Return ONLY a JSON object in this format: "
                "{'Reviewer': '', 'Review': ''}."
        },
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt":
                "Return ONLY a JSON object in this format: "
                "{'Reviewer': '', 'Review': ''}."
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregate all reviewer feedback and provide final improvement suggestions.",
        "max_turns": 1,
    },
]

# Critic automatically routes feedback to all reviewers
critic.register_nested_chats(review_chats, trigger=writer)


# =====================================================================
# 🔁 Run second pass: Critic + Writer + Reviewers
# =====================================================================
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

# Print final combined summary
print(res.summary)


Critic (to Writer):


Write a concise, engaging blog post (with a title) about DeepLearning.AI.
The entire post must be under 100 words.


--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering enthusiasts and professionals alike. Founded by Andrew Ng, it offers comprehensive courses that cover everything from neural networks to machine learning applications. With a hands-on approach and industry-relevant projects, learners can build practical skills to leverage AI in real-world scenarios. The community fosters collaboration, connecting aspiring AI developers across the globe. Join DeepLearning.AI to enhance your knowledge and stay ahead in the rapidly-evolving world of AI technology. Embrace the future of innovation today!

--------------------------------------------------------------------------------
Cri